In [3]:
import glob
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import seaborn as sns
from scipy.stats import linregress
from dask.diagnostics import ProgressBar

# Need to find large ensemble for Precip and Temperature

In [22]:
canesm = ['CanESM5_r10i1p1f1', 'CanESM5_r10i1p2f1',
       'CanESM5_r1i1p1f1', 'CanESM5_r1i1p2f1', 'CanESM5_r2i1p1f1',
       'CanESM5_r2i1p2f1', 'CanESM5_r3i1p1f1', 'CanESM5_r3i1p2f1',
       'CanESM5_r4i1p1f1', 'CanESM5_r4i1p2f1', 'CanESM5_r5i1p1f1',
       'CanESM5_r5i1p2f1', 'CanESM5_r6i1p1f1', 'CanESM5_r6i1p2f1',
       'CanESM5_r7i1p1f1', 'CanESM5_r7i1p2f1', 'CanESM5_r8i1p1f1',
       'CanESM5_r8i1p2f1', 'CanESM5_r9i1p1f1']
accessesm1 = ['ACCESS-ESM1-5_r10i1p1f1',
       'ACCESS-ESM1-5_r11i1p1f1', 'ACCESS-ESM1-5_r12i1p1f1',
       'ACCESS-ESM1-5_r13i1p1f1', 'ACCESS-ESM1-5_r14i1p1f1',
       'ACCESS-ESM1-5_r15i1p1f1', 'ACCESS-ESM1-5_r16i1p1f1',
       'ACCESS-ESM1-5_r17i1p1f1', 'ACCESS-ESM1-5_r18i1p1f1',
       'ACCESS-ESM1-5_r19i1p1f1', 'ACCESS-ESM1-5_r1i1p1f1',
       'ACCESS-ESM1-5_r20i1p1f1', 'ACCESS-ESM1-5_r21i1p1f1',
       'ACCESS-ESM1-5_r22i1p1f1', 'ACCESS-ESM1-5_r23i1p1f1',
       'ACCESS-ESM1-5_r24i1p1f1', 'ACCESS-ESM1-5_r25i1p1f1',
       'ACCESS-ESM1-5_r26i1p1f1', 'ACCESS-ESM1-5_r27i1p1f1',
       'ACCESS-ESM1-5_r28i1p1f1', 'ACCESS-ESM1-5_r29i1p1f1',
       'ACCESS-ESM1-5_r2i1p1f1', 'ACCESS-ESM1-5_r30i1p1f1',
       'ACCESS-ESM1-5_r31i1p1f1', 'ACCESS-ESM1-5_r32i1p1f1',
       'ACCESS-ESM1-5_r33i1p1f1', 'ACCESS-ESM1-5_r34i1p1f1',
       'ACCESS-ESM1-5_r35i1p1f1', 'ACCESS-ESM1-5_r36i1p1f1',
       'ACCESS-ESM1-5_r37i1p1f1', 'ACCESS-ESM1-5_r38i1p1f1',
       'ACCESS-ESM1-5_r39i1p1f1', 'ACCESS-ESM1-5_r3i1p1f1',
       'ACCESS-ESM1-5_r40i1p1f1', 'ACCESS-ESM1-5_r4i1p1f1',
       'ACCESS-ESM1-5_r5i1p1f1', 'ACCESS-ESM1-5_r6i1p1f1',
       'ACCESS-ESM1-5_r7i1p1f1', 'ACCESS-ESM1-5_r8i1p1f1',
       'ACCESS-ESM1-5_r9i1p1f1']

list_of_gcms = ['CNRM-CM6-1_r1i1p1f2', 'UKESM1-0-LL_r1i1p1f2',
       'GISS-E2-1-G_r1i1p1f2', 'IITM-ESM_r1i1p1f1', 'CanESM5_r1i1p1f1',
       'INM-CM5-0_r1i1p1f1', 'INM-CM4-8_r1i1p1f1', 'TaiESM1_r1i1p1f1',
       'EC-Earth3-Veg-LR_r1i1p1f1', 'EC-Earth3_r1i1p1f1',
       'EC-Earth3-Veg_r1i1p1f1', 'CMCC-ESM2_r1i1p1f1', 'CMCC-CM2-SR5_r1i1p1f1',
       'ACCESS-ESM1-5_r1i1p1f1', 'ACCESS-CM2_r4i1p1f1', 'IPSL-CM6A-LR_r1i1p1f1',
       'NorESM2-LM_r1i1p1f1', 'NorESM2-MM_r1i1p1f1', 'MPI-ESM1-2-LR_r1i1p1f1',
       'MPI-ESM1-2-HR_r1i1p1f1']

# Updated this script to produce absolute anomalies here

In [ ]:
# We need to use the historical and concatenate the future on here
# we also should use the "list_of_gcms" list here. 

In [21]:
def get_list_of_files(varname = "tasmax", GCM_variant_list = list_of_gcms,  ssp = 'ssp370', base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public'):
    gcm = [GCM.split('_')[0] for GCM in GCM_variant_list]
    variant = [GCM.split('_')[1] for GCM in GCM_variant_list]
    if not varname == "pr":
        if ssp == 'historical':
            files_glob_tx = [glob.glob(f'{base_path}/{ssp}/daily/{varname}/{varname}_CMIP_*_{gcm[i]}_{ssp}_{variant[i]}_day_NZ12km_NIWA-REMS_{varname}_v??????.nc')[0] for i in range(len(gcm))]
        else:
            files_glob_tx = [glob.glob(f'{base_path}/{ssp}/daily/{varname}/{varname}_ScenarioMIP_*_{gcm[i]}_{ssp}_{variant[i]}_day_NZ12km_NIWA-REMS_{varname}_v??????.nc')[0] for i in range(len(gcm))]
    else:
        if ssp == 'historical':
            files_glob_tx = [glob.glob(f'{base_path}/{ssp}/daily/{varname}/{varname}_CMIP_*_{gcm[i]}_{ssp}_{variant[i]}_day_NZ12km_NIWA-REMS_v??????_pr.nc')[0] for i in range(len(gcm))]
        else:
            files_glob_tx = [glob.glob(f'{base_path}/{ssp}/daily/{varname}/{varname}_ScenarioMIP_*_{gcm[i]}_{ssp}_{variant[i]}_day_NZ12km_NIWA-REMS_v??????_pr.nc')[0] for i in range(len(gcm))]
    return files_glob_tx

In [ ]:
z1 = np.random.rand(0,2,100)

In [23]:
def compute_future_reference(file, txx_reference, seasonal_reference, historical_dset, varname ="tasmax"):
    with ProgressBar():
        with xr.open_dataset(file, chunks={'time': 300}) as ds:
            ds = ds.sel(time=slice("2015", None))
            ds = xr.concat([historical_dset, ds[varname]], dim ="time")
            tasmax = ds

            # Compute TXx more efficiently using Dask
            if varname == "pr":
                txx_future =tasmax.groupby("time.year").max()# 100* (tasmax.groupby("time.year").max() - txx_reference)/txx_reference
            else:
                
                txx_future = tasmax.groupby("time.year").max()# - txx_reference

            # Compute seasonal mean for February (DJF mid-point)
            seasonal = tasmax.resample(time="1MS").mean()
            seasonal = seasonal.rolling(time=3).mean()
            if varname == "pr":
                seasonal = (seasonal.sel(time=seasonal.time.dt.month == 2))# - seasonal_reference)/seasonal_reference
            else:
                
                seasonal = seasonal.sel(time=seasonal.time.dt.month == 2)# - seasonal_reference

            return txx_future.compute(), seasonal.compute()


In [ ]:
ssps = ['ssp126','ssp245', 'ssp370', 'ssp585']
# first get the historical
historical_files = get_list_of_files(varname = "tasmax", GCM_variant_list = list_of_gcms,  ssp = "historical", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp126_files = get_list_of_files(varname = "tasmax", GCM_variant_list = list_of_gcms,  ssp = "ssp126", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp245_files = get_list_of_files(varname = "tasmax", GCM_variant_list = list_of_gcms,  ssp = "ssp245", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp370_files = get_list_of_files(varname = "tasmax", GCM_variant_list = list_of_gcms,  ssp = "ssp370", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp585_files = get_list_of_files(varname = "tasmax", GCM_variant_list = list_of_gcms,  ssp = "ssp585", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
seasonal_dsets = []
txx_dsets = []
for i, file in enumerate(historical_files):
    print(file.split('/')[-1].split('_')[3], ssp126_files[i].split('/')[-1].split('_')[3], ssp245_files[i].split('/')[-1].split('_')[3],ssp370_files[i].split('/')[-1].split('_')[3] ,ssp585_files[i].split('/')[-1].split('_')[3])
    historical_tx = xr.open_dataset(historical_files[i]).sel(time = slice("1960","2014")).tasmax
    # first compute a climatological reference
    txx_reference = historical_tx.groupby('time.year').max().mean("year") # a reference climatology
    seasonal_reference = historical_tx.groupby('time.season').mean().sel(season = 'DJF')
    
    txx_ssp126, seasonal_ssp126 = compute_future_reference(ssp126_files[i], txx_reference, seasonal_reference, historical_tx)
    txx_ssp245, seasonal_ssp245 = compute_future_reference(ssp245_files[i], txx_reference, seasonal_reference, historical_tx)
    txx_ssp370, seasonal_ssp370 = compute_future_reference(ssp370_files[i], txx_reference, seasonal_reference, historical_tx)
    txx_ssp585, seasonal_ssp585 = compute_future_reference(ssp585_files[i], txx_reference, seasonal_reference, historical_tx)
    
    txx_dset = xr.concat([txx_ssp126, txx_ssp245, txx_ssp370, txx_ssp585], dim = "ssp")
    txx_dset['ssp'] = (('ssp'), ssps)
    seasonal_dset = xr.concat([seasonal_ssp126, seasonal_ssp245, seasonal_ssp370, seasonal_ssp585], dim = "ssp")
    seasonal_dset['ssp'] = (('ssp'), ssps)
    txx_dset = txx_dset.expand_dims({"GCM":1})
    seasonal_dset = seasonal_dset.expand_dims({"GCM":1})

    gcm = ssp126_files[i].split('/')[-1].split('_')[3]
    txx_dset['GCM'] = (('GCM'), [gcm])
    seasonal_dset['GCM'] = (('GCM'), [gcm])
    txx_dsets.append(txx_dset)
    seasonal_dsets.append(seasonal_dset)        

CNRM-CM6-1 CNRM-CM6-1 CNRM-CM6-1 CNRM-CM6-1 CNRM-CM6-1
[########################################] | 100% Completed | 10.75 s
[########################################] | 100% Completed | 6.74 sms
[########################################] | 100% Completed | 9.97 ss
[########################################] | 100% Completed | 6.72 sms
[########################################] | 100% Completed | 10.74 s
[########################################] | 100% Completed | 7.16 ss
[########################################] | 100% Completed | 11.13 s
[########################################] | 100% Completed | 7.19 sms
UKESM1-0-LL UKESM1-0-LL UKESM1-0-LL UKESM1-0-LL UKESM1-0-LL
[########################################] | 100% Completed | 10.31 s
[########################################] | 100% Completed | 6.68 ss
[########################################] | 100% Completed | 11.07 s
[########################################] | 100% Completed | 6.45 ss
[########################################]

In [ ]:
new_dsets = []
for dset in seasonal_dsets:
    dset['time'] = pd.to_datetime(dset.time.dt.strftime("%Y-%m-%d"))
    new_dsets.append(dset)

DJF_mme = xr.concat(new_dsets, dim ="GCM").to_dataset(name = 'DJF_MME')
TXX_mme = xr.concat(txx_dsets, dim ="GCM").to_dataset(name = 'TXX_MME')

In [ ]:


merged_dset_TXX_DJF = xr.merge([TXX_mme, DJF_mme])
merged_dset_TXX_DJF.to_netcdf('/nesi/project/niwa00018/ML_downscaling_CCAM/AI_large_ensemble/internal_variability_data/Multi_Model_Decomposition_tasmax_v160625_raw.nc')

In [ ]:
ssps = ['ssp126','ssp245', 'ssp370', 'ssp585']
# first get the historical
historical_files = get_list_of_files(varname = "pr", GCM_variant_list = list_of_gcms,  ssp = "historical", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp126_files = get_list_of_files(varname = "pr", GCM_variant_list = list_of_gcms,  ssp = "ssp126", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp245_files = get_list_of_files(varname = "pr", GCM_variant_list = list_of_gcms,  ssp = "ssp245", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp370_files = get_list_of_files(varname = "pr", GCM_variant_list = list_of_gcms,  ssp = "ssp370", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp585_files = get_list_of_files(varname = "pr", GCM_variant_list = list_of_gcms,  ssp = "ssp585", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
seasonal_dsets_pr = []
rx1day_dsets_pr = []
for i, file in enumerate(historical_files):
    print(file.split('/')[-1].split('_')[3], ssp126_files[i].split('/')[-1].split('_')[3], ssp245_files[i].split('/')[-1].split('_')[3],ssp370_files[i].split('/')[-1].split('_')[3] ,ssp585_files[i].split('/')[-1].split('_')[3])
    historical_tx = xr.open_dataset(historical_files[i]).sel(time = slice("1960","2014")).pr
    # first compute a climatological reference
    txx_reference = historical_tx.groupby('time.year').max().mean("year") # a reference climatology
    seasonal_reference = historical_tx.groupby('time.season').mean().sel(season = 'DJF')
    
    txx_ssp126, seasonal_ssp126 = compute_future_reference(ssp126_files[i], txx_reference, seasonal_reference,historical_tx,  varname ="pr")
    txx_ssp245, seasonal_ssp245 = compute_future_reference(ssp245_files[i], txx_reference, seasonal_reference,historical_tx, varname ="pr")
    txx_ssp370, seasonal_ssp370 = compute_future_reference(ssp370_files[i], txx_reference, seasonal_reference, historical_tx, varname ="pr")
    txx_ssp585, seasonal_ssp585 = compute_future_reference(ssp585_files[i], txx_reference, seasonal_reference, historical_tx, varname ="pr")
    
    txx_dset = xr.concat([txx_ssp126, txx_ssp245, txx_ssp370, txx_ssp585], dim = "ssp")
    txx_dset['ssp'] = (('ssp'), ssps)
    seasonal_dset = xr.concat([seasonal_ssp126, seasonal_ssp245, seasonal_ssp370, seasonal_ssp585], dim = "ssp")
    seasonal_dset['ssp'] = (('ssp'), ssps)
    txx_dset = txx_dset.expand_dims({"GCM":1})
    seasonal_dset = seasonal_dset.expand_dims({"GCM":1})

    gcm = ssp126_files[i].split('/')[-1].split('_')[3]
    txx_dset['GCM'] = (('GCM'), [gcm])
    seasonal_dset['GCM'] = (('GCM'), [gcm])
    rx1day_dsets_pr.append(txx_dset)
    seasonal_dsets_pr.append(seasonal_dset)        

In [ ]:
Rx1day_mme_pr = xr.concat(rx1day_dsets_pr, dim ="GCM").to_dataset(name = 'Rx1day_MME')

new_dsets = []
for dset in seasonal_dsets_pr:
    dset['time'] = pd.to_datetime(dset.time.dt.strftime("%Y-%m-%d"))
    new_dsets.append(dset)

DJF_mme_pr= xr.concat(new_dsets, dim ="GCM").to_dataset(name = 'DJF_MME_pr')

#DJF_mme_pr = xr.concat(seasonal_dsets_pr, dim ="GCM").to_dataset(name = 'DJF_MME')
merged_dset_RX1DAY_DJF_pr = xr.merge([Rx1day_mme_pr , DJF_mme_pr])
merged_dset_RX1DAY_DJF_pr.to_netcdf('/nesi/project/niwa00018/ML_downscaling_CCAM/AI_large_ensemble/internal_variability_data/Multi_Model_Decomposition_pr_v160625_raw.nc')

In [ ]:
ssps = ['ssp370']
# first get the historical
historical_files = get_list_of_files(varname = "tasmax", GCM_variant_list = accessesm1 + canesm,  ssp = "historical", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp370_files = get_list_of_files(varname = "tasmax", GCM_variant_list = accessesm1 + canesm,  ssp = "ssp370", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
print(len(historical_files))
seasonal_dsets_iv = []
txx_dsets_iv = []
for i, file in enumerate(historical_files):
    historical_tx = xr.open_dataset(historical_files[i]).sel(time = slice("1960","2014")).tasmax
    # first compute a climatological reference
    txx_reference = historical_tx.groupby('time.year').max().mean("year") # a reference climatology
    seasonal_reference = historical_tx.groupby('time.season').mean().sel(season = 'DJF')
    txx_ssp370, seasonal_ssp370 = compute_future_reference(ssp370_files[i], txx_reference, seasonal_reference, historical_tx)
    
    # this is the large ensemble code
    variant = file.split('/')[-1].split('_')[5]
    gcm = ssp370_files[i].split('/')[-1].split('_')[3]
    
    
    txx_dset = txx_ssp370.expand_dims({"GCM_variant":1})
    seasonal_dset = seasonal_ssp370.expand_dims({"GCM_variant":1})
    print(f'{gcm}_{variant}')
    
    txx_dset['GCM_variant'] = (('GCM_variant'), [f'{gcm}_{variant}'])
    txx_dset['GCM'] = (('GCM_variant'), [f'{gcm}'])
    seasonal_dset['GCM_variant'] = (('GCM_variant'), [f'{gcm}_{variant}'])
    seasonal_dset['GCM'] = (('GCM_variant'), [f'{gcm}'])
    txx_dsets_iv.append(txx_dset)
    seasonal_dsets_iv.append(seasonal_dset)


In [ ]:
TXX_IV = xr.concat(txx_dsets_iv, dim ="GCM_variant").to_dataset(name = 'TXX_IV')
new_dsets = []
for dset in seasonal_dsets_iv:
    dset['time'] = pd.to_datetime(dset.time.dt.strftime("%Y-%m-%d"))
    new_dsets.append(dset)
DJF_IV = xr.concat(new_dsets, dim ="GCM_variant").to_dataset(name = 'DJF_IV')


In [99]:
# DJF_IV =  DJF_IV.drop("season").rename({"time":"year"})

In [101]:
# DJF_IV['year'] =  DJF_IV.year.dt.year

In [ ]:
TXX_IV = xr.concat(txx_dsets_iv, dim ="GCM_variant").to_dataset(name = 'TXX_IV')
new_dsets = []
for dset in seasonal_dsets_iv:
    dset['time'] = pd.to_datetime(dset.time.dt.strftime("%Y-%m-%d"))
    new_dsets.append(dset)
DJF_IV = xr.concat(new_dsets, dim ="GCM_variant").to_dataset(name = 'DJF_IV')
#DJF_IV['GCM_variant'] = TXX_IV .GCM_variant.values

merged_dset_TXX_DJF_IV = xr.merge([TXX_IV, DJF_IV])
merged_dset_TXX_DJF_IV.to_netcdf('/nesi/project/niwa00018/ML_downscaling_CCAM/AI_large_ensemble/internal_variability_data/Multi_Model_Decomposition_tasmax_LE_IV_v160625_raw.nc')

In [ ]:
ssps = ['ssp370']
# first get the historical
historical_files = get_list_of_files(varname = "pr", GCM_variant_list = accessesm1 + canesm,  ssp = "historical", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
ssp370_files = get_list_of_files(varname = "pr", GCM_variant_list = accessesm1 + canesm,  ssp = "ssp370", base_path ='/nesi/nobackup/niwa03712/ML_Downscaled_CMIP6/NIWA-REMS_CCAM_public')
print(len(historical_files))
seasonal_dsets_iv_pr = []
txx_dsets_iv_pr = []
for i, file in enumerate(historical_files):
    historical_tx = xr.open_dataset(historical_files[i]).sel(time = slice("1960","2014")).pr
    # first compute a climatological reference
    txx_reference = historical_tx.groupby('time.year').max().mean("year") # a reference climatology
    seasonal_reference = historical_tx.groupby('time.season').mean().sel(season = 'DJF')
    txx_ssp370, seasonal_ssp370 = compute_future_reference(ssp370_files[i], txx_reference, seasonal_reference, historical_tx, varname = "pr")
    
    # this is the large ensemble code
    variant = file.split('/')[-1].split('_')[5]
    gcm = ssp370_files[i].split('/')[-1].split('_')[3]
    
    
    txx_dset = txx_ssp370.expand_dims({"GCM_variant":1})
    seasonal_dset = seasonal_ssp370.expand_dims({"GCM_variant":1})
    print(f'{gcm}_{variant}')
    
    txx_dset['GCM_variant'] = (('GCM_variant'), [f'{gcm}_{variant}'])
    txx_dset['GCM'] = (('GCM_variant'), [f'{gcm}'])
    seasonal_dset['GCM_variant'] = (('GCM_variant'), [f'{gcm}_{variant}'])
    seasonal_dset['GCM'] = (('GCM_variant'), [f'{gcm}'])
    txx_dsets_iv_pr.append(txx_dset)
    seasonal_dsets_iv_pr.append(seasonal_dset)


In [117]:
# TXX_IV = xr.concat(txx_dsets_iv_pr, dim ="GCM_variant").to_dataset(name = 'RX1Day_IV')
# DJF_IV = xr.concat(seasonal_dsets_iv_pr, dim ="GCM_variant").to_dataset(name = 'DJF_IV')
# merged_dset_TXX_DJF_IV_pr = xr.merge([TXX_IV, DJF_IV])
# merged_dset_TXX_DJF_IV_pr.to_netcdf('/nesi/project/niwa00018/ML_downscaling_CCAM/AI_large_ensemble/internal_variability_data/Multi_Model_Decomposition_pr_LE_IV.nc')

In [ ]:
TXX_IV = xr.concat(txx_dsets_iv_pr, dim ="GCM_variant").to_dataset(name = 'RX1Day_IV')
new_dsets = []
for dset in seasonal_dsets_iv_pr:
    dset['time'] = pd.to_datetime(dset.time.dt.strftime("%Y-%m-%d"))
    new_dsets.append(dset)
DJF_IV = xr.concat(new_dsets, dim ="GCM_variant").to_dataset(name = 'DJF_IV_pr')


merged_dset_TXX_DJF_IV = xr.merge([TXX_IV, DJF_IV])
merged_dset_TXX_DJF_IV.to_netcdf('/nesi/project/niwa00018/ML_downscaling_CCAM/AI_large_ensemble/internal_variability_data/Multi_Model_Decomposition_pr_LE_IV_v160625_raw.nc')

In [728]:
concats = xr.concat(full_dset, dim ="ssp")

In [820]:
concats.to_netcdf(r'/nesi/project/niwa00018/ML_downscaling_CCAM/AI_large_ensemble/TXX_internal_variability_smoothing_decomposition.nc')

In [821]:
print(len(files_glob_tx))

77


In [46]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
concat_internal_variability = xr.open_dataset(r'/nesi/project/niwa00018/ML_downscaling_CCAM/AI_large_ensemble/TXX_internal_variability_smoothing_decomposition_contribution.nc')

In [4]:
concat_internal_variability = concat_internal_variability.stack(z =['GCM', 'GCM_variant']).dropna("z")

In [9]:
concat_internal_variability['tasmax_resid_2'] = (('ssp','lat','lon','year','z'), concat_internal_variability.tasmax_resid.transpose("ssp","lat","lon","time","z").values)

In [11]:
concat_internal_variability['tasmax_resid_2'] = concat_internal_variability['tasmax_resid_2'] + concat_internal_variability['tasmax']

In [16]:
var1 = concat_internal_variability['tasmax_resid_2'].squeeze().sel(z = concat_internal_variability.GCM =='ACCESS-ESM1-5').var("z")
var2 = concat_internal_variability['tasmax_resid_2'].squeeze().sel(z = concat_internal_variability.GCM =='CanESM5').var("z")

In [20]:
original_array = xr.open_dataset(r'/nesi/project/niwa00018/ML_downscaling_CCAM/AI_large_ensemble/TXX_internal_variability_smoothing_decomposition.nc')